In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Download Tensorflow

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

## Download the neccessary data into the Colab Instance
We will split our dataset into three subsets: training set, validation set and testing set.

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
import tensorflow_hub as hub
import os

BATCH_SIZE = 32
IMAGE_SIZE = 224

def format_image(image, label):
    image = tf.image.resize(image, (IMAGE_SIZE, IMAGE_SIZE)) / 255.0
    return  image, label

(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'cats_vs_dogs',
    data_dir='/content/dataset/',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    shuffle_files=True, 
    as_supervised=True,
)

# tf.data.experimental.cardinality(raw_train_dataset).numpy()
num_examples = metadata.splits['train'].num_examples
num_classes = metadata.features['label'].num_classes
print("Numbers of images: ", num_examples)
print("Numbers of classes: ", num_classes)

train_batches = raw_train.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = raw_validation.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = raw_test.map(format_image).batch(1)

Shuffling and writing examples to /content/dataset/cats_vs_dogs/4.0.0.incompleteTMIDQJ/cats_vs_dogs-train.tfrecord


Dataset cats_vs_dogs downloaded and prepared to /content/dataset/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.
Numbers of images:  23262
Numbers of classes:  2


## Transfer learning
We download the MobileNet_v2 model and then take the feature extractor from the model. We define our last layer for cat vs dog classification and optimizer for our networks.


In [ ]:
IMAGE_SIZE = (224, 224)
FV_SIZE = 1280
MODULE_HANDLE ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor = hub.KerasLayer(MODULE_HANDLE,
                                   input_shape=IMAGE_SIZE + (3,), 
                                   output_shape=[FV_SIZE],
                                   trainable=False)

model = tf.keras.Sequential([
        feature_extractor,
        tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


Then, we train our networks within only 5 epochs.

In [ ]:
model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

EPOCHS = 5

history = model.fit(train_batches,
                 epochs=EPOCHS,
                 validation_data=validation_batches)

## Run your Tensorflow model

Let's now take a look at actually running a prediction using the model. This code will test 100 images from the testing set

In [ ]:
from tqdm import tqdm
import numpy as np

predictions = []

# This will report how many iterations per second, where each
# iteration is 100 predictions
test_labels, test_imgs = [], []
for img, label in tqdm(test_batches.take(100)):
    predictions.append(model.predict(img))
    
    test_labels.append(label.numpy()[0])
    test_imgs.append(img)


# This will tell you how many of the predictions were correct
score = 0
for item in range(0,len(predictions)):
  prediction=np.argmax(predictions[item])
  label = test_labels[item]
  if prediction==label:
    score=score+1

print("Out of 100 predictions I got " + str(score) + " correct")

## Save the Tensorflow model

In [ ]:
CATS_VS_DOGS_SAVED_MODEL = "./exp_saved_model"
tf.saved_model.save(model, CATS_VS_DOGS_SAVED_MODEL)

In [ ]:
import subprocess

def du(path):
    """disk usage in human readable format (e.g. '2,1GB')"""
    return subprocess.check_output(['du','-sh', path]).split()[0].decode('utf-8')
# Get file size in bytes for a given model
print("Model size in Mb:")
print(du(CATS_VS_DOGS_SAVED_MODEL))

## Convert our model using Tensorflow Lite

We will use Tensorflow Lite to convert our model into a compressed flat buffer, which can be used to deploy on mobile and IoT devices. Here we use post-training quantization (PTQ) and optimization

In [ ]:
import pathlib

TFLITE_PATH = "/content/tflite/"
TFLITE_NAME = 'model1.tflite'
tflite_model_file = TFLITE_PATH+TFLITE_NAME

if os.path.isdir(TFLITE_PATH) is False:
  os.mkdir(TFLITE_PATH)

converter = tf.lite.TFLiteConverter.from_saved_model(CATS_VS_DOGS_SAVED_MODEL)
tflite_model = converter.convert()

open(tflite_model_file, "wb").write(tflite_model)
print("Tensorflow lite model size in byte: ")
du(TFLITE_PATH)
# This will report back the file size in bytes

## Test Tensorflow Lite model

In [ ]:
from tqdm import tqdm
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=tflite_model_file)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

predictions = []

# This will report how many iterations per second, where each
# iteration is 100 predictions
test_labels, test_imgs = [], []
for img, label in tqdm(test_batches.take(100)):
    interpreter.set_tensor(input_index, img)
    interpreter.invoke()
    predictions.append(interpreter.get_tensor(output_index))
    
    test_labels.append(label.numpy()[0])
    test_imgs.append(img)


# This will tell you how many of the predictions were correct
score = 0
for item in range(0,len(predictions)):
  prediction=np.argmax(predictions[item])
  label = test_labels[item]
  if prediction==label:
    score=score+1

print("Out of 100 predictions I got " + str(score) + " correct")


##Exercice 1
How much is the size of the Tensorflow model and those of the Tensorflow Lite model in byte ? \\

Is the accuracy of the Tensorflow Lite model not bad ?



##Exercice 2
We can try some options:

* By default: Hint : converter.optimizations = [tf.lite.Optimize.DEFAULT]
* Float16 Quantization : converter.target_spec.supported_types = [tf.float16]
* Opimization for size: Hint : [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
* Opimization for latency: Hint : [tf.lite.Optimize.OPTIMIZE_FOR_LATENCY]

What can we see in term of the size and the accuracy of models ?

To learn more about post-training quantization and optimization, please check out the user guides at https://www.tensorflow.org/lite/performance/post_training_quantization

## Clean Up

Before running the next exercise, run the following cell to terminate the kernel and free memory resources:

In [ ]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)